In [14]:
import cv2
import numpy as np
import os
from difflib import SequenceMatcher
import Levenshtein
import csv

In [15]:
# Folder to store CSV results
os.makedirs("robustness_results", exist_ok=True)
csv_path = os.path.join("robustness_results", "saltpepper_metrics.csv")

In [16]:
# Initialize CSV file with headers
with open(csv_path, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["File", "PSNR (dB)", "BER", "Sequence Similarity (%)", "Levenshtein Distance", "Levenshtein Similarity (%)", "Extracted Snippet"])

In [17]:
# Fungsi untuk mengubah bit menjadi text
def bits_to_text(bits):
    chars = []
    for i in range(0, len(bits), 8):
        byte = bits[i:i+8]
        chars.append(chr(int(byte, 2)))
    return ''.join(chars)

# Fungsi untuk mengubah text menjadi bit
def text_to_bits(text):
    return ''.join(format(ord(c), '08b') for c in text)

In [18]:
def extract_lsb(image_path):
    # Load image
    img = cv2.imread(image_path)
    if img is None:
        print(f"gagal membaca gambar: {image_path}")
        return None
    
    # Flatten image
    flat = img.flatten()
    total = len(flat)

    # Baca header (32 bit)
    header_bits = ""
    for i in range(32):
        header_bits += str(flat[i] & 1)

    msg_len = int(header_bits, 2)

    # msg_len <= 0, maka tidak valid
    if msg_len <= 0:
        return "(Invalid payload)"

    # kapasitas minimal, yaitu 32 header + msg_len
    if 32 + msg_len > total:
        return "(Invalid payload)"

    bits = ""
    end = 32 + msg_len

    for i in range(32, end):
        bits += str(flat[i] & 1)

    # Jika bits kurang, maka corrupted
    if len(bits) != msg_len:
        return "(Corrupted payload)"

    return bits_to_text(bits)


In [19]:
def psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return 999
    PIXEL_MAX = 255.0
    return 20 * np.log10(PIXEL_MAX / np.sqrt(mse))

In [20]:
# --- Function to add Salt & Pepper noise ---
def add_salt_pepper_noise(image, prob=0.02):
    noisy = np.copy(image)
    total_pixels = image.shape[0] * image.shape[1]
    num_salt = int(total_pixels * prob / 2)
    num_pepper = int(total_pixels * prob / 2)

    # Salt
    coords = [np.random.randint(0, i - 1, num_salt) for i in image.shape[:2]]
    noisy[coords[0], coords[1]] = 255

    # Pepper
    coords = [np.random.randint(0, i - 1, num_pepper) for i in image.shape[:2]]
    noisy[coords[0], coords[1]] = 0

    return noisy

In [21]:
# --- Batch add noise ---
def batch_add_noise(folder_in, folder_out, prob=0.02):
    os.makedirs(folder_out, exist_ok=True)
    for filename in os.listdir(folder_in):
        if filename.lower().endswith(".png"):
            img = cv2.imread(os.path.join(folder_in, filename))
            noisy = add_salt_pepper_noise(img, prob)
            out_path = os.path.join(folder_out, f"noise_saltpepper_{filename}")
            cv2.imwrite(out_path, noisy)
            print("Noisy version saved:", out_path)

In [22]:
# --- Bit Error Rate ---
def bit_error_rate(original_text, extracted_text):
    orig_bits = text_to_bits(original_text)
    extr_bits = text_to_bits(extracted_text)

    min_len = min(len(orig_bits), len(extr_bits))
    errors = sum(1 for i in range(min_len) if orig_bits[i] != extr_bits[i])
    total = len(orig_bits)
    return errors / total if total > 0 else 0

In [23]:
# --- Text Similarity (SequenceMatcher) ---
def text_similarity(a, b):
    return SequenceMatcher(None, a, b).ratio() * 100  # percentage

In [24]:
# --- Levenshtein Distance ---
def levenshtein_metrics(a, b):
    distance = Levenshtein.distance(a, b)
    max_len = max(len(a), len(b))
    normalized = (1 - distance / max_len) * 100 if max_len > 0 else 100
    return distance, normalized

In [25]:
def test_robustness(stego_folder, noisy_folder, original_message, noise_prob=None):
    """
    Compute metrics for images in noisy_folder, print them, save to CSV, and return averages.
    noise_prob: float, the noise probability to store in CSV
    """
    psnr_list, ber_list, seqsim_list, levdist_list, levsim_list = [], [], [], [], []

    for filename in os.listdir(noisy_folder):
        if filename.lower().endswith(".png"):
            noisy_path = os.path.join(noisy_folder, filename)
            ori_name = filename.replace("noise_saltpepper_", "")
            ori_path = os.path.join(stego_folder, ori_name)

            extracted = extract_lsb(noisy_path)

            # --- Metrics ---
            ber = bit_error_rate(original_message, extracted)
            sim_ratio = text_similarity(original_message, extracted)
            lev_dist, lev_sim = levenshtein_metrics(original_message, extracted)

            ori = cv2.imread(ori_path)
            noisy = cv2.imread(noisy_path)
            psnr_val = psnr(ori, noisy)

            # --- Print Proof (unchanged) ---
            print(f"\nFile: {filename}")
            print(f"PSNR: {psnr_val:.2f} dB")
            print(f"Bit Error Rate (BER): {ber:.6f}")
            print(f"Sequence Similarity: {sim_ratio:.2f}%")
            print(f"Levenshtein Distance: {lev_dist} | Similarity: {lev_sim:.2f}%")
            print("Extracted snippet:")
            print("------------------------------------------------------------")
            print(extracted[:200] + "..." if len(extracted) > 200 else extracted)
            print("------------------------------------------------------------\n")

            # --- Save to CSV with Noise Probability ---
            snippet = extracted[:200] + "..." if len(extracted) > 200 else extracted
            with open(csv_path, mode="a", newline="", encoding="utf-8") as file:
                writer = csv.writer(file)
                writer.writerow([
                    filename,
                    noise_prob if noise_prob is not None else "",
                    f"{psnr_val:.2f}",
                    f"{ber:.6f}",
                    f"{sim_ratio:.2f}",
                    lev_dist,
                    f"{lev_sim:.2f}",
                    snippet
                ])

            # --- Store metrics for averaging ---
            psnr_list.append(psnr_val)
            ber_list.append(ber)
            seqsim_list.append(sim_ratio)
            levdist_list.append(lev_dist)
            levsim_list.append(lev_sim)

    # --- Compute averages ---
    num_files = len(psnr_list)
    if num_files > 0:
        return {
            "PSNR": sum(psnr_list)/num_files,
            "BER": sum(ber_list)/num_files,
            "SeqSim": sum(seqsim_list)/num_files,
            "LevDist": sum(levdist_list)/num_files,
            "LevSim": sum(levsim_list)/num_files
        }
    else:
        return None


In [26]:
# --- Run ---
if __name__ == "__main__":
    # Original message
    message = (
        "Batik merupakan salah satu warisan budaya Indonesia yang memiliki nilai sejarah dan filosofi yang sangat tinggi. "
        "Setiap motif batik tidak hanya menghadirkan keindahan visual, tetapi juga membawa pesan dan makna yang mendalam. "
        "Motif parang, misalnya, menggambarkan kekuatan, semangat pantang menyerah, serta keberanian dalam menghadapi tantangan hidup. "
        "Sementara itu, motif kawung dipercaya melambangkan kesucian hati dan keinginan manusia untuk mencapai keseimbangan hidup."
    )

    # Add noise and test
    noise_probs = [0.02, 0.05, 0.1]
    avg_metrics = {}

    for prob in noise_probs:
        folder = f"noise_saltpepper_batik_{prob}"
        batch_add_noise("stego_batik", folder, prob)
        avg_metrics[prob] = test_robustness("stego_batik", folder, message, noise_prob=prob)
    
    print("\n--- AVERAGE METRICS PER NOISE PROBABILITY ---")
    for prob, metrics in avg_metrics.items():
        print(f"Noise {prob}: {metrics}")

Noisy version saved: noise_saltpepper_batik_0.02\noise_saltpepper_stego_1.png
Noisy version saved: noise_saltpepper_batik_0.02\noise_saltpepper_stego_10.png
Noisy version saved: noise_saltpepper_batik_0.02\noise_saltpepper_stego_11.png
Noisy version saved: noise_saltpepper_batik_0.02\noise_saltpepper_stego_12.png
Noisy version saved: noise_saltpepper_batik_0.02\noise_saltpepper_stego_13.png
Noisy version saved: noise_saltpepper_batik_0.02\noise_saltpepper_stego_14.png
Noisy version saved: noise_saltpepper_batik_0.02\noise_saltpepper_stego_15.png
Noisy version saved: noise_saltpepper_batik_0.02\noise_saltpepper_stego_16.png
Noisy version saved: noise_saltpepper_batik_0.02\noise_saltpepper_stego_17.png
Noisy version saved: noise_saltpepper_batik_0.02\noise_saltpepper_stego_18.png
Noisy version saved: noise_saltpepper_batik_0.02\noise_saltpepper_stego_19.png
Noisy version saved: noise_saltpepper_batik_0.02\noise_saltpepper_stego_2.png
Noisy version saved: noise_saltpepper_batik_0.02\noise